In [1]:
# Imports

import pandas as pd
import numpy as np
import time
import plotly.express as px


In [2]:
# Eight Queens State

class EightQueensState:
    def __init__(self, board_size, state=None):
        if state is None:
            # Inital board set up
            self.state = np.random.randint(0, board_size, board_size)
        else:
            self.state = np.array(state) 

        # Set the goal fitness value where there is no queen attacks 
        self.goal_fitness = int(((board_size ** 2) - board_size) / 2)
    
    
    def fitness(self):
        count = 0
        n = len(self.state)
        
        for i in range(n - 1):
            remaining_queens = self.state[i + 1:]
            current_queen = self.state[i]
            
            # Count of the queen attacks on the row 
            count += (current_queen == remaining_queens).sum()
            
            # Count of the queen attacks on the diagonal 
            distances = np.arange(1, n - i)
            upper_diagonal = current_queen + distances
            lower_diagonal = current_queen - distances
            
            # Total queen on queen attacks
            count += sum((remaining_queens == upper_diagonal) | (remaining_queens == lower_diagonal))

        # Return the final fitness 
        return self.goal_fitness - count

In [3]:
# Genetic Algorithm

class GeneticAlgorithm:
    def __init__(self, board_size, population_size, mutation_probability, initial_population='Neighbors'):
        self.board_size = board_size
        self.population_size = population_size
        self.mutation_probability = mutation_probability
        self.last_state = None

        # Set the goal fitness value where there are no queen attacks
        self.goal_fitness = int(((board_size ** 2) - board_size) / 2)
        
        if initial_population == 'Random':
            # Initial population - all random individuals
            self.current_population = [EightQueensState(board_size) for _ in range(population_size)]

        elif initial_population == 'Neighbors':
            # Initial population - unique neighbors of a random initial state
            initial_state = np.random.randint(0, board_size, board_size)
            self.current_population = [EightQueensState(board_size, self.random_neighbour(initial_state)) for _ in range(population_size)]


    def random_neighbour(self, state):
        new_state = state.copy() 

        # Choose a random column
        column = np.random.choice(range(self.board_size)) 

        # Create a list of possible row positions excluding the current row of the queen
        possible_rows = [row for row in range(self.board_size) if row != state[column]]

        # Set the queen in the chosen column to a new row position
        new_state[column] = np.random.choice(possible_rows)

        return new_state


    def calculate_population_fitness(self):
        # Calculate fitness and selection probability for each individual in the population
        population_data = []
        total_fitness = 0
        
        for individual in self.current_population:
            fitness = individual.fitness()
            population_data.append({
                'state': individual.state,
                'fitness': fitness
            })
            total_fitness += fitness
        
        # Calculate selection probability for each individual based on fitness
        for data in population_data:
            data['selection_prob'] = data['fitness'] / total_fitness
            
        # Return the sorted the population by the highest selection probabilities
        return sorted(population_data, key=lambda x: x['selection_prob'], reverse=True)


    def select_parent(self, population_data):
        # Generate a random number between 0 and 1
        random_value = np.random.random()
        
        # Use cumulative probability to select an individual
        cumulative_probability = 0.0
        for individual in population_data:
            cumulative_probability += individual['selection_prob']
            if cumulative_probability > random_value:
                return individual['state']


    def crossover(self, parent1, parent2):
        # Perform a single-point crossover to produce two offspring via randomisation
        idx = np.random.randint(1, len(parent1))  

        # Apply the cross over of two parents to product two children 
        return np.concatenate([parent1[:idx], parent2[idx:]]), np.concatenate([parent2[:idx], parent1[idx:]])


    def mutate(self, state):
        # Randomly mutate a queen's position in one column with a certain probability
        if np.random.random() < self.mutation_probability:

             # Select a random column to mutate
            idx = np.random.randint(len(state))  

            # Randomly choose a new row position for the queen in the selected column, excluding current row 
            state[idx] = np.random.choice(list(set(range(self.board_size)) - {state[idx]}))
        return state


    def create_next_generation(self, population_data):
        # Generate the next generation by selecting parents, applying crossover, and mutating offspring
        next_generation = []
        
        # Fill the rest of the population with offspring
        while len(next_generation) < self.population_size:
            parent1 = self.select_parent(population_data)
            parent2 = self.select_parent(population_data)
            
            # Create two children through crossover
            child1, child2 = self.crossover(parent1, parent2)
            
            # Apply mutation to each child
            child1 = self.mutate(child1)
            child2 = self.mutate(child2)
            
            # Add children to the next generation
            next_generation.extend([
                EightQueensState(self.board_size, child1),
                EightQueensState(self.board_size, child2)
            ])
        
        # Ensure the population size remains constant
        self.current_population = next_generation[:self.population_size]


    def run(self):
        generation = 0
        fitness_history = []
        
        while generation < 5000:
            population_data = self.calculate_population_fitness()
            
            # Check if the best individual in the population is a solution
            if population_data[0]['fitness'] == self.goal_fitness:
                self.last_state = population_data[0]['state']
                # print(f"Solution found in generation {generation}")
                return fitness_history, generation  # Return fitness history and generation where solution is found
            
            # Track the best fitness for each generation (optional for progress analysis)
            best_fitness = population_data[0]['fitness']
            fitness_history.append(best_fitness)
            
            # Generate the next generation
            self.create_next_generation(population_data)
            
            generation += 1

        return fitness_history, generation 
    

    def display_solution(self):
        # Display the chessboard with queens in the positions defined by self.last_state
        for row in range(self.board_size):
            print(f'{self.board_size - row} |', end=' ')
            for col in range(self.board_size):
                if self.last_state[col] == row:
                    print('Q ', end='')  
                else:
                    print('- ', end='') 
            print(f'|')

In [ ]:
# Run multiple trials and collect performance metrics
def run_multiple_trials(trials, initial_population):
    data = []
    
    for trial in range(trials):
        start_time = time.time()
        
        ga = GeneticAlgorithm(
            board_size=8,
            population_size=300,
            mutation_probability=0.15,
            initial_population=initial_population
        )
        
        ga.run()
        
        end_time = time.time()
        time_taken = end_time - start_time
        generation_count = len(ga.current_population)  # Assuming this tracks generations
        
        # Record trial data
        data.append({
            'trial': trial + 1,
            'time_taken': time_taken,
            'generation_count': generation_count,
            'initial_population_type': initial_population
        })
    
    return pd.DataFrame(data)

# Collect data for random and neighbor initial populations
trials = 20
data_random = run_multiple_trials(trials, 'Random')


data_neighbor = run_multiple_trials(trials, 'Neighbors')

# Combine data into one DataFrame for comparison
data = pd.concat([data_random, data_neighbor])

# Calculate summary statistics for time and generation count
summary_stats = data.groupby('initial_population_type').agg(
    time_mean=('time_taken', 'mean'),
    time_std=('time_taken', 'std'),
    generation_mean=('generation_count', 'mean'),
    generation_std=('generation_count', 'std')
).reset_index()

# Plotting the results using Plotly Express
# Time taken per trial for each initial population type
fig_time = px.box(data, x='initial_population_type', y='time_taken', 
                  title="Time Taken for Random vs. Neighbor Initial Populations",
                  labels={'time_taken': 'Time Taken (seconds)', 'initial_population_type': 'Initial Population Type'})

# Generation count per trial for each initial population type
fig_generation = px.box(data, x='initial_population_type', y='generation_count', 
                        title="Generation Count for Random vs. Neighbor Initial Populations",
                        labels={'generation_count': 'Generation Count', 'initial_population_type': 'Initial Population Type'})

# Summary statistics plot
fig_summary = px.bar(summary_stats, x='initial_population_type', y=['time_mean', 'generation_mean'], 
                     error_y=['time_std', 'generation_std'],
                     barmode='group',
                     title="Average Performance with Standard Deviation",
                     labels={'value': 'Average Performance', 'initial_population_type': 'Initial Population Type', 'variable': 'Metric'},
                     facet_col='variable')

# Display plots
fig_time.show()
fig_generation.show()
fig_summary.show()


In [ ]:
# Run

def run_multiple_trials(trails):
    all_times = []
    for trial in range(trails):
        print(f"\nTrial {trial + 1}:")
        
        start_time = time.time()
        
        ga = GeneticAlgorithm(
            board_size=8,
            population_size=300,
            mutation_probability=0.1,
            initial_population='Random'
        )
        
        ga.run()
        
        end_time = time.time()
        time_taken = end_time - start_time
        all_times.append(time_taken)
        
        print(f"Time taken: {time_taken:.2f} seconds")
    
    print(f"Average Time: {(sum(all_times)/len(all_times)):.2f} seconds")


run_multiple_trials(20)


In [ ]:
# Plot GA

def plot_ga(solution, fitness_history):

    # Create DataFrame
    df = pd.DataFrame({
        'Generation': range(len(fitness_history)),
        'Fitness': fitness_history
    })

    # Create the figure with dark theme using plotly express
    fig = px.line(
        df,
        x='Generation',
        y='Fitness',
        title='Eight Queens - Genetic Algorithm Fitness Evolution',
        labels={'Fitness': 'Fitness Score (28 = Perfect Solution)', 'Generation': 'Generation'},
        template='plotly_dark'
    )

    # Add the solution point as a marker if found
    solution_point = df[df['Fitness'] == 28]
    if not solution_point.empty:
        fig.add_scatter(
            x=solution_point['Generation'],
            y=solution_point['Fitness'],
            mode='markers',
            name='Solution Found',
            marker=dict(color='#00FF00', size=15, symbol='star')
        )

    # Add reference line for perfect fitness
    fig.add_hline(
        y=28, 
        line_dash="dash", 
        line_color="rgba(0, 255, 0, 0.3)",
        annotation_text="Perfect Fitness (28)",
        annotation_position="right",
        annotation_font_color="white"
    )

    # Add annotations for initial and final fitness
    fig.add_annotation(
        x=0,
        y=fitness_history[0],
        text=f"Initial Fitness: {fitness_history[0]}",
        showarrow=True,
        arrowhead=1,
        ax=-50,
        ay=-30,
        font=dict(size=14, color='#00B5F7'),
        bgcolor='rgba(50, 50, 50, 0.8)'
    )

    if solution_point.empty:
        final_gen = len(fitness_history) - 1
        final_fitness = fitness_history[-1]
        annotation_text = f"Final Fitness: {final_fitness}"
    else:
        final_gen = solution_point['Generation'].iloc[0]
        annotation_text = "Solution Found!"

    fig.add_annotation(
        x=final_gen,
        y=fitness_history[final_gen],
        text=annotation_text,
        showarrow=True,
        arrowhead=1,
        ax=50,
        ay=-50,
        font=dict(size=14, color='#00FF00'),
        bgcolor='rgba(50, 50, 50, 0.8)'
    )

    # Final adjustments
    fig.update_layout(
        xaxis=dict(gridcolor='rgba(128, 128, 128, 0.2)', zeroline=False),
        yaxis=dict(gridcolor='rgba(128, 128, 128, 0.2)', zeroline=False, range=[min(fitness_history) - 1, 29]),
        height = 700,
        width=1500,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01,
            bgcolor='rgba(50, 50, 50, 0.8)'
        ),
        hovermode='x unified',
        margin=dict(t=100, b=50, l=50, r=50)
    )

    print(f"Solution found in {final_gen + 1} generations")
    print(f"Final fitness: {fitness_history[final_gen]}")
    fig.show()

In [4]:
# Optimisation

import numpy as np
from tqdm import tqdm
import time
from dataclasses import dataclass

@dataclass
class OptimizationResult:
    pop_size: int
    mut_prob: float
    generations: int
    time_taken: float
    initial_population: str

def test_parameters(params) -> OptimizationResult:
    """Run GA with given parameters and return generations needed to find solution"""
    board_size, pop_size, mut_prob, init_pop = params
    start_time = time.time()
    
    ga = GeneticAlgorithm(
        board_size=board_size,
        population_size=pop_size,
        mutation_probability=mut_prob,
        initial_population=init_pop  # Pass initial_population to the GeneticAlgorithm
    )
    
    fitness_history, generations = ga.run()
    time_taken = time.time() - start_time
    
    return OptimizationResult(pop_size, mut_prob, generations, time_taken, init_pop)


def optimize_parameters(board_size=8, trials=10):  # Increased default trials
    """Test different parameter combinations prioritizing consistent performance"""
    # Expanded parameter ranges
    population_sizes = [20, 50, 100, 250, 500]
    mutation_probs = [0.01, 0.03, 0.8, 0.15, 0.3]
    initial_populations = ['Random', 'Neighbors']
    
    results = []
    
    # Create parameter combinations
    params = [(board_size, pop, mut, init_pop) 
             for pop in population_sizes 
             for mut in mutation_probs 
             for init_pop in initial_populations]
    
    # Run trials with progress bar
    total_combinations = len(params) * trials
    with tqdm(total=total_combinations, desc="Testing combinations") as pbar:
        for _ in range(trials):
            for param_set in params:
                result = test_parameters(param_set)
                results.append(result)
                pbar.update(1)
    
    # Process results
    performance = {}
    for result in results:
        key = (result.pop_size, result.mut_prob, result.initial_population)
        if key not in performance:
            performance[key] = {'gens': [], 'times': []}
        performance[key]['gens'].append(result.generations)
        performance[key]['times'].append(result.time_taken)
    
    # Calculate performance metrics with emphasis on consistency
    avg_performance = {}
    for k, v in performance.items():
        gen_array = np.array(v['gens'])
        time_array = np.array(v['times'])
        success_mask = gen_array < 1000
        
        if sum(success_mask) > 0:  # Only consider combinations with some successes
            successful_gens = gen_array[success_mask]
            successful_times = time_array[success_mask]
            
            # Calculate metrics
            success_rate = sum(success_mask) / len(gen_array)
            avg_gens = np.mean(successful_gens)
            std_gens = np.std(successful_gens)
            cv_gens = std_gens / avg_gens if avg_gens > 0 else float('inf')  # Coefficient of variation
            avg_time = np.mean(successful_times)
            max_time = np.max(successful_times)
            
            # Consistency score (lower is better)
            consistency_score = (cv_gens * 0.4 +                     # Weight variation heavily
                               (1 - success_rate) * 0.3 +            # Weight success rate
                               (max_time / 30) * 0.3)                # Penalize any runs over 30s
            
            avg_performance[k] = {
                'avg_gens': avg_gens,
                'std_gens': std_gens,
                'cv_gens': cv_gens,
                'success_rate': success_rate,
                'avg_time': avg_time,
                'max_time': max_time,
                'consistency_score': consistency_score
            }
    
    # Sort by consistency score (lower is better)
    sorted_params = sorted(avg_performance.items(), 
                         key=lambda x: x[1]['consistency_score'])
    
    return sorted_params

# Run optimization
print("Optimizing parameters...")
best_params = optimize_parameters(trials=10)  # Increased trials for better statistics

print("\nTop 5 Parameter Combinations:")
print("Pop Size | Mut Prob | Init Pop  | Avg Gens | CV Gens | Success % | Avg Time | Max Time | Consistency")
print("-" * 110)
for (pop_size, mut_prob, init_pop), metrics in best_params[:5]:
    print(f"{pop_size:^8d} | {mut_prob:^8.2f} | {init_pop:^9} | {metrics['avg_gens']:^8.1f} | "
          f"{metrics['cv_gens']:^7.2f} | {metrics['success_rate']*100:^8.1f}% | "
          f"{metrics['avg_time']:^8.2f}s | {metrics['max_time']:^8.2f}s | "
          f"{metrics['consistency_score']:^10.3f}")

# Print detailed recommendation
best_pop, best_mut, best_init_pop = best_params[0][0]
best_metrics = best_params[0][1]
print("\nRecommended Parameters:")
print(f"Population Size: {best_pop}")
print(f"Mutation Probability: {best_mut}")
print(f"Initial Population: {best_init_pop}")
print(f"Expected Performance:")
print(f"- Success Rate: {best_metrics['success_rate']*100:.1f}%")
print(f"- Average Generations: {best_metrics['avg_gens']:.1f}")
print(f"- Coefficient of Variation: {best_metrics['cv_gens']:.2f}")
print(f"- Average Time: {best_metrics['avg_time']:.2f}s")
print(f"- Maximum Time: {best_metrics['max_time']:.2f}s")


Optimizing parameters...


Testing combinations: 100%|██████████| 500/500 [1:04:39<00:00,  7.76s/it]


Top 5 Parameter Combinations:
Pop Size | Mut Prob | Init Pop  | Avg Gens | CV Gens | Success % | Avg Time | Max Time | Consistency
--------------------------------------------------------------------------------------------------------------
  250    |   0.30   | Neighbors |  227.1   |  0.42   |   90.0  % |   2.57  s |   4.62  s |   0.246   
  250    |   0.15   | Neighbors |  142.2   |  0.54   |  100.0  % |   1.59  s |   3.11  s |   0.247   
  250    |   0.30   |  Random   |  263.3   |  0.51   |  100.0  % |   3.00  s |   6.49  s |   0.270   
   20    |   0.01   |  Random   |   20.0   |  0.00   |   10.0  % |   0.02  s |   0.02  s |   0.270   
   20    |   0.03   |  Random   |  165.0   |  0.00   |   10.0  % |   0.15  s |   0.15  s |   0.271   

Recommended Parameters:
Population Size: 250
Mutation Probability: 0.3
Initial Population: Neighbors
Expected Performance:
- Success Rate: 90.0%
- Average Generations: 227.1
- Coefficient of Variation: 0.42
- Average Time: 2.57s
- Maximum Time: 4

In [ ]:
# Extended Work

# Add elitism - keep the best individual from the current generation
# next_generation.extend([EightQueensState(self.board_size, individual['state']) for individual in population_data[:self.elitism_size]])


In [5]:
# Simpler optimisation 


def test_parameters(board_size, population_size, mutation_probability):

    start_time = time.time()
    
    ga = GeneticAlgorithm(
        board_size=board_size,
        population_size=population_size,
        mutation_probability=mutation_probability
    )
    
    _, generations = ga.run()
    time_taken = time.time() - start_time
    
    return OptimizationResult(pop_size, mut_prob, generations, time_taken)

def optimize_parameters(board_size=8, trials=5):
    """Find best parameters through simple grid search"""
    # Simple parameter ranges
    population_sizes = [50, 100, 250]
    mutation_probs = [0.1, 0.15, 0.2]
    
    best_avg_generations = float('inf')
    best_params = None
    best_performance = None
    
    print("Testing parameter combinations...")
    
    # Test each combination
    for pop_size in population_sizes:
        for mut_prob in mutation_probs:
            generations_list = []
            times_list = []
            
            # Run multiple trials
            for _ in range(trials):
                result = test_parameters(board_size, pop_size, mut_prob)
                generations_list.append(result.generations)
                times_list.append(result.time_taken)
            
            # Calculate averages
            avg_generations = np.mean(generations_list)
            avg_time = np.mean(times_list)
            max_time = max(times_list)
            
            # Update best if better found
            if avg_generations < best_avg_generations:
                best_avg_generations = avg_generations
                best_params = (pop_size, mut_prob)
                best_performance = {
                    'avg_generations': avg_generations,
                    'avg_time': avg_time,
                    'max_time': max_time
                }
            
            print(f"Pop Size: {pop_size}, Mut Prob: {mut_prob:.2f}")
            print(f"Avg Generations: {avg_generations:.1f}")
            print(f"Avg Time: {avg_time:.2f}s")
            print("-" * 40)
    
    # Print final recommendation
    print("\nRecommended Parameters:")
    print(f"Population Size: {best_params[0]}")
    print(f"Mutation Probability: {best_params[1]}")
    print(f"\nExpected Performance:")
    print(f"Average Generations: {best_performance['avg_generations']:.1f}")
    print(f"Average Time: {best_performance['avg_time']:.2f}s")
    print(f"Maximum Time: {best_performance['max_time']:.2f}s")
    
    return best_params

# Run optimization
best_params = optimize_parameters()

Testing parameter combinations...
Pop Size: 50, Mut Prob: 0.10
Avg Generations: 1068.2
Avg Time: 2.05s
----------------------------------------
Pop Size: 50, Mut Prob: 0.15
Avg Generations: 715.6
Avg Time: 1.38s
----------------------------------------
Pop Size: 50, Mut Prob: 0.20
Avg Generations: 537.2
Avg Time: 1.05s
----------------------------------------
Pop Size: 100, Mut Prob: 0.10
Avg Generations: 254.0
Avg Time: 1.01s
----------------------------------------
Pop Size: 100, Mut Prob: 0.15
Avg Generations: 432.2
Avg Time: 1.73s
----------------------------------------
Pop Size: 100, Mut Prob: 0.20
Avg Generations: 323.2
Avg Time: 1.28s
----------------------------------------
Pop Size: 250, Mut Prob: 0.10
Avg Generations: 255.6
Avg Time: 2.81s
----------------------------------------
Pop Size: 250, Mut Prob: 0.15
Avg Generations: 87.6
Avg Time: 0.97s
----------------------------------------
Pop Size: 250, Mut Prob: 0.20
Avg Generations: 229.8
Avg Time: 2.55s
--------------------